### -1- Active player scrape
* Scraping capfriendly for active player data
* First, we scrap the entire pool of "active" players
* Then, we scrape by team for maximal coverage
* We finish this section off by appending them and removing any dups

#### -1.1- All active players

In [243]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
import time
from bs4 import BeautifulSoup
from datetime import datetime
import re

try:
    # Specify the path to Chromedriver (replace with your actual path)
    chromedriver_path = 'C:/Users/domen/Downloads/chromedriver_win32/chromedriver.exe'

    # Create Chrome options
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')  # Optional: Run headless (without opening a browser window)

    # Initialize the WebDriver
    driver = webdriver.Chrome(options=chrome_options)

    # URL pattern for the pages
    url_pattern = 'https://www.capfriendly.com/browse/active?stats-season=2023&age-calculation-date=today&display=signing-team,draft,signing-status,expiry-year,performance-bonus,signing-bonus,caphit-percent,aav,length,minors-salary,base-salary,type,signing-age,signing-date,arbitration,extension&hide=position,handed,skater-stats,goalie-stats&pg={}'

    # Initialize an empty DataFrame to store the data
    all_data = pd.DataFrame()

    # Loop through all pages (32 pages)
    for page_number in range(1, 33):
        try:
            # Load the page in the WebDriver
            driver.get(url_pattern.format(page_number))

            # Wait for the page to load (you may need to adjust the sleep time)
            time.sleep(5)  # Wait for 5 seconds to allow the page to load

            # Get the HTML content of the page
            page_source = driver.page_source

            # Parse the HTML content with BeautifulSoup
            soup = BeautifulSoup(page_source, 'html.parser')

            # Find the table containing the data
            table = soup.find('table')

            # Read the table data into a DataFrame
            data = pd.read_html(str(table))[0]

            # Concatenate the data to the main DataFrame
            all_data = pd.concat([all_data, data], ignore_index=True)

            print(f"Page {page_number} scraped successfully.")
        except Exception as e:
            print(f"Error while scraping page {page_number}: {e}")

    # Close the WebDriver
    driver.quit()

    # Create copy of data
    cap_data = all_data.copy()

    # Split the 'DRAFTED' column into four new columns using regular expressions
    draft_split = cap_data['DRAFTED'].str.extract(r'(\d+) - Round (\d+) - (\d{4}) \(([^)]+)\)')
    draft_split.columns = ['DRAFT_ROUND', 'DRAFT_PICK', 'DRAFT_YEAR', 'DRAFT_TEAM']

    # Add the new columns to your DataFrame
    cap_data = pd.concat([cap_data, draft_split], axis=1)

    # Fix the player name
    cap_data['PLAYER'] = cap_data['PLAYER'].str.split('.').str[1:].str.join('').str.strip()

    # Function to attempt parsing in different date formats
    def parse_date(date_str):
        formats = ["%b. %d, %Y", "%b %d, %Y", "%b. %d, %Y", "%b %d, %Y"]
        for date_format in formats:
            try:
                return pd.to_datetime(date_str, format=date_format).strftime("%b. %d, %Y")  # Format as string
            except ValueError:
                pass
        return None

    # Function to format values
    def format_value(value):
        # Remove '$' and ',' and convert to float
        value_float = float(value.replace('$', '').replace(',', ''))
        if value_float >= 1000000:
            return f"${value_float / 1000000:.2f}M"
        else:
            return f"${value_float / 1000:.1f}K"

    # Format column names to lowercase and replace spaces and periods with underscores
    cap_data.columns = [col.lower().replace('.', '_').replace(' ', '_').replace('__', '_') for col in cap_data.columns]

    # Replace checkmark with 1 or 0
    columns_with_checkmark = cap_data.columns[cap_data.isin(['✔']).any()]
    for column in columns_with_checkmark:
        cap_data[column] = cap_data[column].apply(lambda x: 1 if x == '✔' else 0)

    # Function to replace special characters found in player_full_name
    def replace_special_chars(input_str):
        special_chars = {
            'á': 'a', 'Á': 'A', 'ä': 'a', 'Ä': 'A', 'é': 'e', 'É': 'E', 'è': 'e', 'È': 'E', 'ê': 'e', 'Ê': 'E',
            'í': 'i', 'Í': 'I', 'ï': 'i', 'Ï': 'I', 'ó': 'o', 'Ó': 'O', 'ö': 'o', 'Ö': 'O', 'ô': 'o', 'Ô': 'O',
            'ú': 'u', 'Ú': 'U', 'ü': 'u', 'Ü': 'U', 'û': 'u', 'Û': 'U', 'ñ': 'n', 'Ñ': 'N', 'ç': 'c', 'Ç': 'C',
            'ß': 'ss', 'Æ': 'AE', 'æ': 'ae', 'Ø': 'O', 'ø': 'o', 'ł': 'l', 'Ł': 'L', 'ń': 'n', 'Ń': 'N',
            'ś': 's', 'Ś': 'S', 'ć': 'c', 'Ć': 'C', 'ź': 'z', 'Ź': 'Z', 'ż': 'z', 'Ż': 'Z', 'ğ': 'g', 'Ğ': 'G',
            'şı': 'si', 'Ş': 'Si', 'ķ': 'k', 'Ķ': 'K', 'š': 's', 'Š': 'S', 'č': 'c', 'Č': 'C', 'ž': 'z', 'Ž': 'Z',
            'ň': 'n', 'Ň': 'N', 'ř': 'r', 'Ř': 'R', 'ý': 'y', 'Ý': 'Y', 'ů': 'u', 'Ů': 'U', 'ţ': 't', 'Ţ': 'T',
            'ă': 'a', 'Ă': 'A', 'ş': 's', 'Ş': 'S', 'ď': 'd', 'Ď': 'D', 'ř': 'r', 'Ř': 'R', 'ť': 't', 'Ť': 'T',
            'ĺ': 'l', 'Ĺ': 'L', 'ć': 'c', 'Ć': 'C', 'đ': 'd', 'Đ': 'D', 'ŕ': 'r', 'Ŕ': 'R', 'ľ': 'l', 'Ľ': 'L',
            'ŝ': 's', 'Ŝ': 'S', 'ĥ': 'h', 'Ĥ': 'H', 'ĵ': 'j', 'Ĵ': 'J', 'ŵ': 'w', 'Ŵ': 'W', 'ŷ': 'y', 'Ŷ': 'Y',
            'ẑ': 'z', 'Ẑ': 'Z', 'ơ': 'o', 'Ơ': 'O', 'ī': 'i', 'Ī': 'I', 'ū': 'u', 'Ū': 'U', 'ț': 't', 'Ț': 'T',
            'ș': 's', 'Ș': 'S'
        }
        # Use a regular expression to match and replace special characters
        pattern = re.compile("|".join(map(re.escape, special_chars.keys())))
        return pattern.sub(lambda match: special_chars[match.group(0)], input_str)

    # Assuming you have a DataFrame named cap_data
    cap_data['player_full_name'] = cap_data['player'].apply(replace_special_chars)

    # Apply date parsing to "SIGNING DATE" column
    cap_data['etl_insert_ts'] = pd.Timestamp(datetime.now())
    cap_data['years_left'] = cap_data['exp_year'] - cap_data['etl_insert_ts'].dt.year.astype(int)
    cap_data['cap_hit_format'] = cap_data['cap_hit'].apply(format_value)
    cap_data['aav_format'] = cap_data['aav'].apply(format_value)
    cap_data['active_flag'] = 1

    # Assuming df is your DataFrame and order is the desired column order
    desired_order = [
        'player_full_name', 'active_flag', 'team', 'age', 'signing_team', 'type', 'extension',
        'arb_elig', 'arb_req', 'clause', 'signing', 'expiry', 'salary', 'base_salary',
        'minors', 's_bonus', 'p_bonus', 'drafted', 'draft_round', 'draft_pick',
        'draft_year', 'draft_team', 'signing_age', 'signing_date', 'length',
        'exp_year', 'years_left', 'cap_hit', 'cap_hit_format', 'cap_hit_%', 'aav',
        'aav_format', 'etl_insert_ts']

    # Reorder the columns
    cap_data_clean = cap_data[desired_order]
    print("Active player data scraped and stored as a pandas dataframe: cap_data_clean.")

except Exception as e:
    print(f"Error: {e}")

<ipython-input-243-b88c026a47c5>:45: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(str(table))[0]


Page 1 scraped successfully.


<ipython-input-243-b88c026a47c5>:45: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(str(table))[0]


Page 2 scraped successfully.


<ipython-input-243-b88c026a47c5>:45: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(str(table))[0]


...

Page 30 scraped successfully.


<ipython-input-243-b88c026a47c5>:45: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(str(table))[0]


Page 31 scraped successfully.


<ipython-input-243-b88c026a47c5>:45: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(str(table))[0]


Page 32 scraped successfully.
Active player data scraped and stored as a pandas dataframe: cap_data_clean.


#### -1.2- Active players by team

In [244]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
import time
from bs4 import BeautifulSoup
from datetime import datetime
import re

try:
    # Specify the path to Chromedriver (replace with your actual path)
    chromedriver_path = 'C:/Users/domen/Downloads/chromedriver_win32/chromedriver.exe'

    # Create Chrome options
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')  # Optional: Run headless (without opening a browser window)

    # Initialize the WebDriver
    driver = webdriver.Chrome(options=chrome_options)

    # nhl team list
    nhl_teams = ["ducks","coyotes","bruins","sabres","flames","hurricanes","blackhawks","avalanche","bluejackets","stars","redwings","oilers","panthers","kings","wild","canadiens","predators","devils","islanders","rangers","senators","flyers","penguins","sharks","kraken","blues","lightning","mapleleafs","canucks","goldenknights","capitals","jets"]

    # URL pattern for the pages
    url_pattern = 'https://www.capfriendly.com/browse/active/2024/caphit/{}?display=signing-team,draft,signing-status,expiry-year,performance-bonus,signing-bonus,caphit-percent,aav,length,minors-salary,base-salary,type,signing-age,signing-date,arbitration,extension&hide=position,handed,skater-stats,goalie-stats'

    # Initialize an empty DataFrame to store the data
    all_data = pd.DataFrame()

    # Loop through all nhl_teams
    for team in nhl_teams:
        try:
            # Load the page in the WebDriver
            driver.get(url_pattern.format(team))

            # Wait for the page to load (you may need to adjust the sleep time)
            time.sleep(5)  # Wait for 5 seconds to allow the page to load

            # Get the HTML content of the page
            page_source = driver.page_source

            # Parse the HTML content with BeautifulSoup
            soup = BeautifulSoup(page_source, 'html.parser')

            # Find the table containing the data
            table = soup.find('table')

            # Read the table data into a DataFrame
            data = pd.read_html(str(table))[0]

            # Concatenate the data to the main DataFrame
            all_data = pd.concat([all_data, data], ignore_index=True)

            print(f"Team {team} scraped successfully.")
        except Exception as e:
            print(f"Error while scraping team {team}: {e}")

    # Close the WebDriver
    driver.quit()

    # Create copy of data
    team_cap_data = all_data.copy()

    # Split the 'DRAFTED' column into four new columns using regular expressions
    draft_split = team_cap_data['DRAFTED'].str.extract(r'(\d+) - Round (\d+) - (\d{4}) \(([^)]+)\)')
    draft_split.columns = ['DRAFT_ROUND', 'DRAFT_PICK', 'DRAFT_YEAR', 'DRAFT_TEAM']

    # Add the new columns to your DataFrame
    team_cap_data = pd.concat([team_cap_data, draft_split], axis=1)

    # Fix the player name
    team_cap_data['PLAYER'] = team_cap_data['PLAYER'].str.split('.').str[1:].str.join('').str.strip()

    # Function to attempt parsing in different date formats
    def parse_date(date_str):
        formats = ["%b. %d, %Y", "%b %d, %Y", "%b. %d, %Y", "%b %d, %Y"]
        for date_format in formats:
            try:
                return pd.to_datetime(date_str, format=date_format).strftime("%b. %d, %Y")  # Format as string
            except ValueError:
                pass
        return None

    # Function to format values
    def format_value(value):
        # Remove '$' and ',' and convert to float
        value_float = float(value.replace('$', '').replace(',', ''))
        if value_float >= 1000000:
            return f"${value_float / 1000000:.2f}M"
        else:
            return f"${value_float / 1000:.1f}K"

    # Format column names to lowercase and replace spaces and periods with underscores
    team_cap_data.columns = [col.lower().replace('.', '_').replace(' ', '_').replace('__', '_') for col in team_cap_data.columns]

    # Replace checkmark with 1 or 0
    columns_with_checkmark = team_cap_data.columns[team_cap_data.isin(['✔']).any()]
    for column in columns_with_checkmark:
        team_cap_data[column] = team_cap_data[column].apply(lambda x: 1 if x == '✔' else 0)

    # Function to replace special characters found in player_full_name
    def replace_special_chars(input_str):
        special_chars = {
            'á': 'a', 'Á': 'A', 'ä': 'a', 'Ä': 'A', 'é': 'e', 'É': 'E', 'è': 'e', 'È': 'E', 'ê': 'e', 'Ê': 'E',
            'í': 'i', 'Í': 'I', 'ï': 'i', 'Ï': 'I', 'ó': 'o', 'Ó': 'O', 'ö': 'o', 'Ö': 'O', 'ô': 'o', 'Ô': 'O',
            'ú': 'u', 'Ú': 'U', 'ü': 'u', 'Ü': 'U', 'û': 'u', 'Û': 'U', 'ñ': 'n', 'Ñ': 'N', 'ç': 'c', 'Ç': 'C',
            'ß': 'ss', 'Æ': 'AE', 'æ': 'ae', 'Ø': 'O', 'ø': 'o', 'ł': 'l', 'Ł': 'L', 'ń': 'n', 'Ń': 'N',
            'ś': 's', 'Ś': 'S', 'ć': 'c', 'Ć': 'C', 'ź': 'z', 'Ź': 'Z', 'ż': 'z', 'Ż': 'Z', 'ğ': 'g', 'Ğ': 'G',
            'şı': 'si', 'Ş': 'Si', 'ķ': 'k', 'Ķ': 'K', 'š': 's', 'Š': 'S', 'č': 'c', 'Č': 'C', 'ž': 'z', 'Ž': 'Z',
            'ň': 'n', 'Ň': 'N', 'ř': 'r', 'Ř': 'R', 'ý': 'y', 'Ý': 'Y', 'ů': 'u', 'Ů': 'U', 'ţ': 't', 'Ţ': 'T',
            'ă': 'a', 'Ă': 'A', 'ş': 's', 'Ş': 'S', 'ď': 'd', 'Ď': 'D', 'ř': 'r', 'Ř': 'R', 'ť': 't', 'Ť': 'T',
            'ĺ': 'l', 'Ĺ': 'L', 'ć': 'c', 'Ć': 'C', 'đ': 'd', 'Đ': 'D', 'ŕ': 'r', 'Ŕ': 'R', 'ľ': 'l', 'Ľ': 'L',
            'ŝ': 's', 'Ŝ': 'S', 'ĥ': 'h', 'Ĥ': 'H', 'ĵ': 'j', 'Ĵ': 'J', 'ŵ': 'w', 'Ŵ': 'W', 'ŷ': 'y', 'Ŷ': 'Y',
            'ẑ': 'z', 'Ẑ': 'Z', 'ơ': 'o', 'Ơ': 'O', 'ī': 'i', 'Ī': 'I', 'ū': 'u', 'Ū': 'U', 'ț': 't', 'Ț': 'T',
            'ș': 's', 'Ș': 'S'
        }
        # Use a regular expression to match and replace special characters
        pattern = re.compile("|".join(map(re.escape, special_chars.keys())))
        return pattern.sub(lambda match: special_chars[match.group(0)], input_str)

    # Assuming you have a DataFrame named team_cap_data
    team_cap_data['player_full_name'] = team_cap_data['player'].apply(replace_special_chars)

    # Apply date parsing to "SIGNING DATE" column
    team_cap_data['etl_insert_ts'] = pd.Timestamp(datetime.now())
    team_cap_data['years_left'] = team_cap_data['exp_year'] - team_cap_data['etl_insert_ts'].dt.year.astype(int)
    team_cap_data['cap_hit_format'] = team_cap_data['cap_hit'].apply(format_value)
    team_cap_data['aav_format'] = team_cap_data['aav'].apply(format_value)
    team_cap_data['active_flag'] = 1

    # Assuming df is your DataFrame and order is the desired column order
    desired_order = [
        'player_full_name', 'active_flag', 'team', 'age', 'signing_team', 'type', 'extension',
        'arb_elig', 'arb_req', 'clause', 'signing', 'expiry', 'salary', 'base_salary',
        'minors', 's_bonus', 'p_bonus', 'drafted', 'draft_round', 'draft_pick',
        'draft_year', 'draft_team', 'signing_age', 'signing_date', 'length',
        'exp_year', 'years_left', 'cap_hit', 'cap_hit_format', 'cap_hit_%', 'aav',
        'aav_format', 'etl_insert_ts']

    # Reorder the columns
    team_cap_data_clean = team_cap_data[desired_order]
    print("Active player data scraped and stored as a pandas dataframe: team_cap_data_clean.")

except Exception as e:
    print(f"Error: {e}")

<ipython-input-244-628f35b9a17a>:48: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(str(table))[0]


Team ducks scraped successfully.


<ipython-input-244-628f35b9a17a>:48: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(str(table))[0]


Team coyotes scraped successfully.


<ipython-input-244-628f35b9a17a>:48: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(str(table))[0]


...

Team goldenknights scraped successfully.


<ipython-input-244-628f35b9a17a>:48: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(str(table))[0]


Team capitals scraped successfully.


<ipython-input-244-628f35b9a17a>:48: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(str(table))[0]


Team jets scraped successfully.
Active player data scraped and stored as a pandas dataframe: team_cap_data_clean.


#### -1.3- Union together

In [245]:
import pandas as pd

# Assuming you have cap_data_clean and team_cap_data_clean DataFrames

# Concatenate the two DataFrames
active_cap_data_clean = pd.concat([cap_data_clean, team_cap_data_clean])

# Sort the DataFrame by 'exp_year' in descending order
active_cap_data_clean.sort_values(by='exp_year', ascending=False, inplace=True)

# Remove duplicate records based on player_full_name and team, keeping the one with the highest 'exp_year'
active_cap_data_clean.drop_duplicates(subset=['player_full_name', 'team'], keep='first', inplace=True)

# Reset the index if needed
active_cap_data_clean.reset_index(drop=True, inplace=True)

### -2- Free agent player scrape
* Scraping capfriendly for free agent data

In [246]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
import time
from bs4 import BeautifulSoup
from datetime import datetime
import re

try:
    # Specify the path to Chromedriver (replace with your actual path)
    chromedriver_path = 'C:/Users/domen/Downloads/chromedriver_win32/chromedriver.exe'

    # Create Chrome options
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')  # Optional: Run headless (without opening a browser window)

    # Initialize the WebDriver
    driver = webdriver.Chrome(options=chrome_options)

    # URL pattern for the pages
    url_pattern = 'https://www.capfriendly.com/browse/free-agents?stats-season=2023&age-calculation-date=today&display=signing-team,draft,signing-status,expiry-year,performance-bonus,signing-bonus,caphit-percent,aav,length,minors-salary,base-salary,type,signing-age,signing-date,arbitration,extension&hide=position,handed,skater-stats,goalie-stats&pg={}'

    # Initialize an empty DataFrame to store the data
    all_data = pd.DataFrame()

    # Loop through all pages (2 pages)
    for page_number in range(1, 3):
        try:
            # Load the page in the WebDriver
            driver.get(url_pattern.format(page_number))

            # Wait for the page to load (you may need to adjust the sleep time)
            time.sleep(5)  # Wait for 5 seconds to allow the page to load

            # Get the HTML content of the page
            page_source = driver.page_source

            # Parse the HTML content with BeautifulSoup
            soup = BeautifulSoup(page_source, 'html.parser')

            # Find the table containing the data
            table = soup.find('table')

            # Read the table data into a DataFrame
            data = pd.read_html(str(table))[0]

            # Concatenate the data to the main DataFrame
            all_data = pd.concat([all_data, data], ignore_index=True)

            print(f"Page {page_number} scraped successfully.")
        except Exception as e:
            print(f"Error while scraping page {page_number}: {e}")

    # Close the WebDriver
    driver.quit()

    # Create copy of data
    fa_cap_data = all_data.copy()

    # Split the 'DRAFTED' column into four new columns using regular expressions
    draft_split = fa_cap_data['DRAFTED'].str.extract(r'(\d+) - Round (\d+) - (\d{4}) \(([^)]+)\)')
    draft_split.columns = ['DRAFT_ROUND', 'DRAFT_PICK', 'DRAFT_YEAR', 'DRAFT_TEAM']

    # Add the new columns to your DataFrame
    fa_cap_data = pd.concat([fa_cap_data, draft_split], axis=1)

    # Fix the player name
    fa_cap_data['PLAYER'] = fa_cap_data['PLAYER'].str.split('.').str[1:].str.join('').str.strip()

    # Function to attempt parsing in different date formats
    def parse_date(date_str):
        formats = ["%b. %d, %Y", "%b %d, %Y", "%b. %d, %Y", "%b %d, %Y"]
        for date_format in formats:
            try:
                return pd.to_datetime(date_str, format=date_format).strftime("%b. %d, %Y")  # Format as string
            except ValueError:
                pass
        return None

    # Function to format values
    def format_value(value):
        # Remove '$' and ',' and convert to float
        value_float = float(value.replace('$', '').replace(',', ''))
        if value_float >= 1000000:
            return f"${value_float / 1000000:.2f}M"
        else:
            return f"${value_float / 1000:.1f}K"

    # Format column names to lowercase and replace spaces and periods with underscores
    fa_cap_data.columns = [col.lower().replace('.', '_').replace(' ', '_').replace('__', '_') for col in fa_cap_data.columns]

    # Replace checkmark with 1 or 0
    columns_with_checkmark = fa_cap_data.columns[fa_cap_data.isin(['✔']).any()]
    for column in columns_with_checkmark:
        fa_cap_data[column] = fa_cap_data[column].apply(lambda x: 1 if x == '✔' else 0)

    # Function to replace special characters found in player_full_name
    def replace_special_chars(input_str):
        special_chars = {
            'á': 'a', 'Á': 'A', 'ä': 'a', 'Ä': 'A', 'é': 'e', 'É': 'E', 'è': 'e', 'È': 'E', 'ê': 'e', 'Ê': 'E',
            'í': 'i', 'Í': 'I', 'ï': 'i', 'Ï': 'I', 'ó': 'o', 'Ó': 'O', 'ö': 'o', 'Ö': 'O', 'ô': 'o', 'Ô': 'O',
            'ú': 'u', 'Ú': 'U', 'ü': 'u', 'Ü': 'U', 'û': 'u', 'Û': 'U', 'ñ': 'n', 'Ñ': 'N', 'ç': 'c', 'Ç': 'C',
            'ß': 'ss', 'Æ': 'AE', 'æ': 'ae', 'Ø': 'O', 'ø': 'o', 'ł': 'l', 'Ł': 'L', 'ń': 'n', 'Ń': 'N',
            'ś': 's', 'Ś': 'S', 'ć': 'c', 'Ć': 'C', 'ź': 'z', 'Ź': 'Z', 'ż': 'z', 'Ż': 'Z', 'ğ': 'g', 'Ğ': 'G',
            'şı': 'si', 'Ş': 'Si', 'ķ': 'k', 'Ķ': 'K', 'š': 's', 'Š': 'S', 'č': 'c', 'Č': 'C', 'ž': 'z', 'Ž': 'Z',
            'ň': 'n', 'Ň': 'N', 'ř': 'r', 'Ř': 'R', 'ý': 'y', 'Ý': 'Y', 'ů': 'u', 'Ů': 'U', 'ţ': 't', 'Ţ': 'T',
            'ă': 'a', 'Ă': 'A', 'ş': 's', 'Ş': 'S', 'ď': 'd', 'Ď': 'D', 'ř': 'r', 'Ř': 'R', 'ť': 't', 'Ť': 'T',
            'ĺ': 'l', 'Ĺ': 'L', 'ć': 'c', 'Ć': 'C', 'đ': 'd', 'Đ': 'D', 'ŕ': 'r', 'Ŕ': 'R', 'ľ': 'l', 'Ľ': 'L',
            'ŝ': 's', 'Ŝ': 'S', 'ĥ': 'h', 'Ĥ': 'H', 'ĵ': 'j', 'Ĵ': 'J', 'ŵ': 'w', 'Ŵ': 'W', 'ŷ': 'y', 'Ŷ': 'Y',
            'ẑ': 'z', 'Ẑ': 'Z', 'ơ': 'o', 'Ơ': 'O', 'ī': 'i', 'Ī': 'I', 'ū': 'u', 'Ū': 'U', 'ț': 't', 'Ț': 'T',
            'ș': 's', 'Ș': 'S'
        }
        # Use a regular expression to match and replace special characters
        pattern = re.compile("|".join(map(re.escape, special_chars.keys())))
        return pattern.sub(lambda match: special_chars[match.group(0)], input_str)

    # Assuming you have a DataFrame named fa_cap_data
    fa_cap_data['player_full_name'] = fa_cap_data['player'].apply(replace_special_chars)

    # Apply date parsing to "SIGNING DATE" column
    fa_cap_data['etl_insert_ts'] = pd.Timestamp(datetime.now())
    fa_cap_data['years_left'] = fa_cap_data['exp_year'] - fa_cap_data['etl_insert_ts'].dt.year.astype(int)
    fa_cap_data['cap_hit_format'] = fa_cap_data['cap_hit'].apply(format_value)
    fa_cap_data['aav_format'] = fa_cap_data['aav'].apply(format_value)
    fa_cap_data['active_flag'] = 0

    # Assuming df is your DataFrame and order is the desired column order
    desired_order = [
        'player_full_name', 'active_flag', 'team', 'age', 'signing_team', 'type', 'extension',
        'arb_elig', 'arb_req', 'clause', 'signing', 'expiry', 'salary', 'base_salary',
        'minors', 's_bonus', 'p_bonus', 'drafted', 'draft_round', 'draft_pick',
        'draft_year', 'draft_team', 'signing_age', 'signing_date', 'length',
        'exp_year', 'years_left', 'cap_hit', 'cap_hit_format', 'cap_hit_%', 'aav',
        'aav_format', 'etl_insert_ts']

    # Reorder the columns
    fa_cap_data_clean = fa_cap_data[desired_order]
    print("Free Agent player data scraped and stored as a pandas dataframe: fa_cap_data_clean.")

except Exception as e:
    print(f"Error: {e}")

<ipython-input-246-acd9e46c46b4>:45: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(str(table))[0]


Page 1 scraped successfully.


<ipython-input-246-acd9e46c46b4>:45: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(str(table))[0]


Page 2 scraped successfully.
Free Agent player data scraped and stored as a pandas dataframe: fa_cap_data_clean.


### -3- Union & Log data

In [247]:
import pandas as pd

# Append the dataframes together, then write as CSV locally
all_player_data_clean = pd.concat([active_cap_data_clean, fa_cap_data_clean])
all_player_data_raw = pd.concat([active_cap_data_clean, fa_cap_data])

# Save the data to a CSV file
all_player_data_raw.to_csv("capfriendly_data_raw.csv", index=False) 
all_player_data_clean.to_csv("capfriendly_data_clean.csv", index=False) 
print(f"Wrote the datasets as CSVs")

Wrote the datasets as CSVs


### -4- Explore data

In [140]:
import re

# Read ranks_data
ranks_data_raw = pd.read_csv("../bq_results/202202_player_ranks.csv")
ranks_data = ranks_data_raw[ranks_data_raw['season_window'] == 'Last 3 seasons']

# Function to replace special characters found in player_full_name
def replace_special_chars(input_str):
  special_chars = {
      'á': 'a', 'Á': 'A', 'ä': 'a', 'Ä': 'A', 'é': 'e', 'É': 'E', 'è': 'e', 'È': 'E', 'ê': 'e', 'Ê': 'E',
      'í': 'i', 'Í': 'I', 'ï': 'i', 'Ï': 'I', 'ó': 'o', 'Ó': 'O', 'ö': 'o', 'Ö': 'O', 'ô': 'o', 'Ô': 'O',
      'ú': 'u', 'Ú': 'U', 'ü': 'u', 'Ü': 'U', 'û': 'u', 'Û': 'U', 'ñ': 'n', 'Ñ': 'N', 'ç': 'c', 'Ç': 'C',
      'ß': 'ss', 'Æ': 'AE', 'æ': 'ae', 'Ø': 'O', 'ø': 'o', 'ł': 'l', 'Ł': 'L', 'ń': 'n', 'Ń': 'N',
      'ś': 's', 'Ś': 'S', 'ć': 'c', 'Ć': 'C', 'ź': 'z', 'Ź': 'Z', 'ż': 'z', 'Ż': 'Z', 'ğ': 'g', 'Ğ': 'G',
      'şı': 'si', 'Ş': 'Si', 'ķ': 'k', 'Ķ': 'K', 'š': 's', 'Š': 'S', 'č': 'c', 'Č': 'C', 'ž': 'z', 'Ž': 'Z',
      'ň': 'n', 'Ň': 'N', 'ř': 'r', 'Ř': 'R', 'ý': 'y', 'Ý': 'Y', 'ů': 'u', 'Ů': 'U', 'ţ': 't', 'Ţ': 'T',
      'ă': 'a', 'Ă': 'A', 'ş': 's', 'Ş': 'S', 'ď': 'd', 'Ď': 'D', 'ř': 'r', 'Ř': 'R', 'ť': 't', 'Ť': 'T',
      'ĺ': 'l', 'Ĺ': 'L', 'ć': 'c', 'Ć': 'C', 'đ': 'd', 'Đ': 'D', 'ŕ': 'r', 'Ŕ': 'R', 'ľ': 'l', 'Ľ': 'L',
      'ŝ': 's', 'Ŝ': 'S', 'ĥ': 'h', 'Ĥ': 'H', 'ĵ': 'j', 'Ĵ': 'J', 'ŵ': 'w', 'Ŵ': 'W', 'ŷ': 'y', 'Ŷ': 'Y',
      'ẑ': 'z', 'Ẑ': 'Z', 'ơ': 'o', 'Ơ': 'O', 'ī': 'i', 'Ī': 'I', 'ū': 'u', 'Ū': 'U', 'ț': 't', 'Ț': 'T',
      'ș': 's', 'Ș': 'S'
  }
    
  # Use a regular expression to match and replace special characters
  pattern = re.compile("|".join(map(re.escape, special_chars.keys())))
  return pattern.sub(lambda match: special_chars[match.group(0)], input_str)

# Assuming you have a DataFrame named fa_cap_data
ranks_data['player_name'] = ranks_data['player_name'].apply(replace_special_chars)

# Write back as CSV
ranks_data.to_csv("ranks_data.csv", index=False)

<ipython-input-140-0dbb9df5527c>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ranks_data['player_name'] = ranks_data['player_name'].apply(replace_special_chars)


In [152]:
import pandas as pd

# Function to format player names
def format_player_name(player_name):
    name_parts = player_name.split()
    if len(name_parts) >= 2:
        first_initial = name_parts[0][0].upper()
        last_name = name_parts[-1]
        formatted_name = first_initial + ' ' + last_name
    else:
        formatted_name = player_name.replace('.', '')  # Remove dots if present
    return formatted_name

# Sample data for df1 and df2 (replace with your actual DataFrames)
df1 = ranks_data.copy()
df2 = pd.read_csv("capfriendly_data_clean.csv")

# Make a second player name key to join
df1['join_player_name'] = df1['player_name'].apply(format_player_name)
df2['join_player_name'] = df2['player_full_name'].apply(format_player_name)

# Left join df1 to df2 on "player_name" = "player_full_name" and "team_code" = "team_code"
merged_df1 = pd.merge(df1, df2[['player_full_name', 'team']], how='left', left_on=['player_name', 'current_team_code'], right_on=['player_full_name', 'team'], suffixes=('', '_1'))
matched_df1 = merged_df1[merged_df1['player_full_name'].notnull()]
unmatched_df1 = merged_df1[merged_df1['player_full_name'].isnull()]

# Left join df1 to df2 on "player_name" = "player_full_name" and "team_code" = "team_code"
merged_df2 = pd.merge(unmatched_df1, df2[['player_full_name']], how='left', left_on=['player_name'], right_on = ['player_full_name'], suffixes=('', '_2'))
matched_df2 = merged_df2[merged_df2['player_full_name_2'].notnull()]
unmatched_df2 = merged_df2[merged_df2['player_full_name_2'].isnull()]

# Left join df1 to df2 on "player_name" = "player_full_name" and "team_code" = "team_code"
merged_df3 = pd.merge(unmatched_df2, df2[['join_player_name', 'player_full_name']], how='left', left_on=['join_player_name'], right_on = ['join_player_name'], suffixes=('', '_3'))
matched_df3 = merged_df3[merged_df3['player_full_name_3'].notnull()]
unmatched_df3 = merged_df3[merged_df3['player_full_name_3'].isnull()]

# Print it all out
print("Total number of players:", df1["player_id"].nunique())
print("... after joining on name + team_code, we get +", matched_df1["player_id"].nunique())
print("... after joining on name (alone), we get +", matched_df2["player_id"].nunique())
print("... after joining on join_name (alone), we get +", matched_df3["player_id"].nunique())
print("... in the end, we are left with ", unmatched_df3["player_id"].nunique())

Total number of players: 937
... after joining on name + team_code, we get + 571
... after joining on name (alone), we get + 222
... after joining on join_name (alone), we get + 36
... in the end, we are left with  108
